<a href="https://colab.research.google.com/github/VigneshNaveen1311/MentalHealthProfile/blob/main/DeBERTa_SPeDaC_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/VigneshNaveen1311/Privacy-Preserved-Mental-health-Profile.git

Cloning into 'Privacy-Preserved-Mental-health-Profile'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 26 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (26/26), 7.95 MiB | 5.53 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
import pandas as pd

# Define the path to the TSV file
tsv_file_path = '/content/Privacy-Preserved-Mental-health-Profile/SPeDaC_Dataset/SPeDaC/SPeDaC1/trainingSENS/CURATION_USER.tsv'

# Initialize lists to hold the parsed data
metadata = []
annotations = []

# Read the file
with open(tsv_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Parse the file
for line in lines:
    line = line.strip()
    # Skip metadata lines
    if line.startswith('#'):
        metadata.append(line)
        continue
    # Process annotation lines
    if line:
        parts = line.split('\t')
        # Assuming the structure is consistent with the provided example
        annotations.append({
            'TokenId': parts[0],
            'CharacterOffsets': parts[1],
            'Token': parts[2],
            'Label': parts[3]
        })

# Convert the annotations list to a DataFrame
df_annotations = pd.DataFrame(annotations)

# Display the DataFrame
print(df_annotations)


         TokenId CharacterOffsets      Token             Label
0            1-1              0-8   Training                 _
1            2-1            10-13        The  Non-sensitive[1]
2            2-2            14-20     Tories  Non-sensitive[1]
3            2-3            21-24        are  Non-sensitive[1]
4            2-4            25-33   believed  Non-sensitive[1]
...          ...              ...        ...               ...
282930  10685-17  1386898-1386901        any  Sensitive[10682]
282931  10685-18  1386902-1386911  political  Sensitive[10682]
282932  10685-19  1386912-1386917      party  Sensitive[10682]
282933  10685-20  1386918-1386919          .  Sensitive[10682]
282934  10685-21  1386920-1386921          "  Sensitive[10682]

[282935 rows x 4 columns]


In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DebertaTokenizer, DebertaForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score

In [5]:
# Define your dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, annotations, tokenizer, max_len):
        self.annotations = annotations
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, item):
        text = self.annotations[item]['Token']
        label = 0 if self.annotations[item]['Label'] == 'Non-sensitive' else 1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [6]:
# Function to read and parse the TSV file
def load_dataset(file_path, tokenizer, max_len):
    annotations = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    for line in lines:
        line = line.strip()
        if line.startswith('#'):  # Skip metadata
            continue
        if line:  # Process annotation lines
            parts = line.split('\t')
            annotations.append({
                'TokenId': parts[0],
                'CharacterOffsets': parts[1],
                'Token': parts[2],
                'Label': parts[3]
            })
    return TextClassificationDataset(annotations=annotations, tokenizer=tokenizer, max_len=max_len)


In [7]:
# Initialize the tokenizer
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')

# Load your training and validation datasets
training_file_path = '/content/Privacy-Preserved-Mental-health-Profile/SPeDaC_Dataset/SPeDaC/SPeDaC1/trainingSENS/CURATION_USER.tsv'
validation_file_path = '/content/Privacy-Preserved-Mental-health-Profile/SPeDaC_Dataset/SPeDaC/SPeDaC1/validationSENS/CURATION_USER.tsv'

max_len = 256
train_dataset = load_dataset(training_file_path, tokenizer, max_len)
val_dataset = load_dataset(validation_file_path, tokenizer, max_len)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

# Initialize the DeBERTa model
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=2)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.weight', 'classifier.bias', 'classifier.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Training loop
model.train()
for epoch in range(3):
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        model.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask)
        loss = torch.nn.functional.cross_entropy(outputs.logits, labels)

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed.")

    # Validation loop
    model.eval()
    val_accuracy = []
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)

        preds = torch.argmax(outputs.logits, dim=1)
        acc = accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())
        val_accuracy.append(acc)

    avg_val_accuracy = sum(val_accuracy) / len(val_accuracy)
    print(f"Validation accuracy: {avg_val_accuracy}")




In [ ]:
# Save the model after training
model_save_path = 'D:\\Spedac model'  # Replace with your desired save directory
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


In [ ]:
# Save the model after training
model_save_path = 'your_model_directory'  # Replace with your desired save directory
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# prompt: connect with git repo

!git init
!git remote add origin https://github.com/VigneshNaveen1311/Privacy-Preserved-Mental-health-Profile.git
!git pull origin main


In [ ]:
# prompt: git commit this file

!git add .
!git commit -m "Trained DeBERTa model and saved it to directory"


In [ ]:
# # Set the path to the TSV file within the cloned repository
# tsv_file_path = '/content/Privacy-Preserved-Mental-health-Profile/SPeDaC_Dataset/SPeDaC/SPeDaC1/trainingSENS/CURATION_USER.tsv'

# # Load the TSV file into a pandas DataFrame
# import pandas as pd
# df = pd.read_csv(tsv_file_path, delimiter='\t', header=0)

# # Verify that it loaded correctly
# print(df.head())

In [ ]:
# import requests
# import pandas as pd

# # URL to the raw TSV file in the GitHub repository
# tsv_file_url = 'https://raw.githubusercontent.com/VigneshNaveen1311/Privacy-Preserved-Mental-health-Profile/main/SPeDaC_Dataset/SPeDaC/SPeDaC1/trainingSENS/CURATION_USER.tsv?token=GHSAT0AAAAAACNXK6KSSPFVFPWKJI5LWSIYZN6QMAQ'
# # Make a request to get the content of the TSV file
# response = requests.get(tsv_file_url)
# response.raise_for_status()  # This will raise an exception for HTTP errors

# # Read the content into a pandas DataFrame
# df = pd.read_csv(pd.compat.StringIO(response.text), delimiter='\t')


In [ ]:
# import pandas as pd
# import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers import DebertaTokenizer, DebertaForSequenceClassification, AdamW
# from sklearn.feature_extraction.text import CountVectorizer
# from torch.nn import BCEWithLogitsLoss

# # Define your dataset class
# class TextClassificationDataset(Dataset):
#     def __init__(self, texts, labels, tokenizer, max_len):
#         self.texts = texts
#         self.labels = labels
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, item):
#         text = str(self.texts[item])
#         label = self.labels[item]
#         encoding = self.tokenizer.encode_plus(
#             text,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             return_token_type_ids=False,
#             padding='max_length',
#             truncation=True,
#             return_attention_mask=True,
#             return_tensors='pt',
#         )
#         return {
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten(),
#             'labels': torch.tensor(label, dtype=torch.long)
#         }

# # Load the dataset
# def load_dataset(file_path, tokenizer, max_len):
#     df = pd.read_csv(file_path, delimiter='\t', header=0)
#     texts = df['text_column_name'].tolist()  # Replace with your text column name
#     labels = df['label_column_name'].tolist() # Replace with your label column name
#     return TextClassificationDataset(texts=texts, labels=labels, tokenizer=tokenizer, max_len=max_len)

# # Initialize the tokenizer
# tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')

# # Load your dataset
# dataset_file_path = 'path_to_your_tsv_file.tsv'  # Replace with the path to your TSV file
# max_len = 256
# dataset = load_dataset(dataset_file_path, tokenizer, max_len)

# # Create a DataLoader
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# # Initialize the DeBERTa model
# model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=2)

# # Use GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Optimizer and loss function
# optimizer = AdamW(model.parameters(), lr=1e-5)
# loss_fn = BCEWithLogitsLoss()

# # Training loop
# model.train()
# for epoch in range(3):
#     for batch in dataloader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         model.zero_grad()

#         outputs = model(input_ids, attention_mask=attention_mask)
#         loss = loss_fn(outputs.logits, labels.float())  # Assuming labels are 0 or 1

#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch+1} completed.")

# # Save the model after training
# model.save_pretrained('your_model_directory')  # Replace with your desired save directory
# tokenizer.save_pretrained('your_model_directory')  # Replace with your desired save directory

# # Note: You should add validation and adjust the loss function for your specific task.
